# Feature Selection

In [ ]:
#import libraries

In [ ]:
import pandas as pd
import numpy as np
import random
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

#user specified packages from 
import EncoderFactory
from DatasetManager import DatasetManager

#IBM packages 
from aix360.algorithms.rbm import FeatureBinarizer
from aix360.algorithms.rbm import LogisticRuleRegression

In [ ]:
#TERMINOLOGY
bucket_encoding = "agg"

dataset_ref_to_datasets = {
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)],
}

encoding_dict = {
    "agg": ["static", "agg"]
}

train_ratio = 0.8
random_state = 22

## Dataset BPIC2015_2_f2

In [ ]:
dataset_ref = 'bpic2015_2_f2'
params_dir = './params_dir'
results_dir = './results' 
cls_encoding = 'agg'
gap = 1
n_iter = 1

method_name = "%s_%s"%(cls_method, cls_encoding)

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]

In [ ]:
# print dataset name

In [ ]:
for dataset_name in datasets:
    print('Dataset:', dataset_name)

In [ ]:
# read the data

In [ ]:
dataset_manager = DatasetManager(dataset_name)
data = dataset_manager.read_dataset()
cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

In [ ]:
#dimensions of the data

In [ ]:
print('#columns:',len(data.columns))
print('#rows:',data.shape[0])

In [ ]:
#data types

In [ ]:
data.dtypes

In [ ]:
#data description

The event log BPIC2015 assembles event logs from the second Dutch municipality pertaining to the building permit. After performing trace prefixing, trace cutting (max. length of 40) and sequence encoding, the training amount of the event log reduced from 41,202 to 22,221 with a final total of 391 deduced columns.

### original dataset

In [ ]:
#original dataset
#this is useful for the original activity names

In [ ]:
import pm4py
log = pm4py.read_xes('BPIC15_2.xes')

In [ ]:
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
dataframe = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)

In [ ]:
pd.set_option("display.max_columns", dataframe.columns.shape[0])
pd.set_option("display.max_rows", dataframe.index.shape[0])

In [ ]:
#show the original activities names next to the new activity names

In [ ]:
dataframe.groupby(['concept:name','activityNameEN']).size()

In [ ]:
#there is only one activity for that one

In [ ]:
len(set(dataframe[dataframe['concept:name']=='08_AWB45_010']['activityNameEN']))

In [ ]:
dataframe[dataframe['concept:name']=='08_AWB45_010']['activityNameEN'].head(1)

### correlation

#### dataset

In [ ]:
#check the correlation with Pearson's correlation for the dataset

In [ ]:
data2 = data.copy()

In [ ]:
data2.head()

In [ ]:
data2.label.unique()

In [ ]:
data2["label"].replace({"regular": 0, "deviant": 1}, inplace=True)

In [ ]:
data2.label.unique()

In [ ]:
#Using Pearson Correlation
plt.figure(figsize=(12,10))
cor = data2.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#correlation with label

In [ ]:
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.4]
relevant_features

In [ ]:
data.shape[0]

In [ ]:
data.shape[1]

#### transformed dataset

In [ ]:
# determine min and max (truncated) prefix lengths

In [ ]:
min_prefix_length = 1
if "traffic_fines" in dataset_name:
    max_prefix_length = 10
elif "bpic2017" in dataset_name:
    max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
else:
    max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

In [ ]:
max_prefix_length

In [ ]:
# split into training and test
train, test = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
if gap > 1:
    outfile = os.path.join(results_dir, "performance_results_%s_%s_%s_gap%s.csv" % (cls_method, dataset_name, method_name, gap))
else:
    outfile = os.path.join(results_dir, "performance_results_%s_%s_%s.csv" % (cls_method, dataset_name, method_name))

In [ ]:
#prefix generation of training data

In [ ]:
dt_train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length) 
#get the label of the train set
train_y = dataset_manager.get_label_numeric(dt_train_prefixes)   

In [ ]:
len(dt_train_prefixes)

In [ ]:
#transform train dataset and add the column names back to the dataframe
feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])
feature_combiner.fit(dt_train_prefixes, train_y)
dt_train_named = feature_combiner.transform(dt_train_prefixes)
dt_train_named = pd.DataFrame(dt_train_named)
names= feature_combiner.get_feature_names()
dt_train_named.columns = names

In [ ]:
#concat them back to correlate correlations

In [ ]:
train_y = pd.DataFrame(train_y)
train_y  = train_y.rename(columns={train_y.columns[0]:'label'})
dt_train_named2 = pd.concat([dt_train_named,pd.DataFrame(train_y)], axis=1)

In [ ]:
pd.set_option('display.max_rows', None)
dt_train_named2.dtypes

##### correlations (>0.5 with label)

In [ ]:
#check the correlations

In [ ]:
#Using Pearson Correlation
cor = dt_train_named2.corr()

In [ ]:
#correlation with label

In [ ]:
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>.5].sort_values(ascending=False)
relevant_features

In [ ]:
#this is the dataframe with the features you want to check if they are correlated with eachother

In [ ]:
activity_correlations = dt_train_named2[['agg__Activity_08_AWB45_010',
'agg__Activity_08_AWB45_020_1',   
'agg__Activity_08_AWB45_020_2',    
'agg__Activity_08_AWB45_030',      
'agg__Activity_08_AWB45_040','label']]

In [ ]:
corr = activity_correlations.corr()

In [ ]:
corr

In [ ]:
#these features need to be uncorrelated with eachother, so drop all but one

In [ ]:
dt_train_named3 = dt_train_named2.drop(columns=['agg__Activity_08_AWB45_010','agg__Activity_08_AWB45_020_1','agg__Activity_08_AWB45_030','agg__Activity_08_AWB45_040'])

##### correlations (>0.3 with label)

In [ ]:
cor = dt_train_named3.corr()

In [ ]:
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.3].sort_values(ascending=False)
relevant_features

In [ ]:
correlations2 = dt_train_named3[['static__Milieu (vergunning)',
'static__Responsible_actor_4634935',   
'agg__Activity_08_AWB45_020_2',    
'agg__Activity_08_AWB45_025',   
'agg__monitoringResource_4634935',
'agg__question_28' ]]

In [ ]:
cor = correlations2.corr()

In [ ]:
cor[cor>0.6]

In [ ]:
#correlates with static_Milieu (vergunning)
dt_train_named4 = dt_train_named3.drop(columns=['static__Responsible_actor_4634935'])

##### correlations (>0.25 with label)

In [ ]:
cor = dt_train_named4.corr()

In [ ]:
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.25].sort_values(ascending=False)
relevant_features

In [ ]:
correlations3 = dt_train_named4[[
'static__Milieu (vergunning)',
'agg__Activity_01_HOOFD_090',                                
'agg__Activity_08_AWB45_020_2',    
'agg__Activity_08_AWB45_025',   
'agg__monitoringResource_4634935',
'agg__question_28',
'agg__question_Uitgebreid']]

In [ ]:
cor = correlations3.corr()

In [ ]:
cor[cor>0.6]

In [ ]:
dt_train_named5 = dt_train_named4.drop(columns=['agg__Activity_01_HOOFD_090'])

##### correlations (>0.20 with label)

In [ ]:
cor = dt_train_named5.corr()

In [ ]:
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.20].sort_values(ascending=False)
relevant_features.sort_values(ascending=False)

In [ ]:
#A lot of relevant features, so you better put these in a list

In [ ]:
columns = list(relevant_features.index)

In [ ]:
correlations4 = dt_train_named5[columns]

In [ ]:
cor = correlations4.corr()

In [ ]:
cor[cor>0.6]

In [ ]:
#The activity 330 stays, the 2 others need to be dropped
dt_train_named6 = dt_train_named5.drop(columns=['agg__Activity_01_HOOFD_370','agg__Activity_01_HOOFD_195','agg__Activity_01_HOOFD_375','agg__Activity_01_HOOFD_380','agg__Activity_01_HOOFD_430','agg__Activity_09_AH_I_010'])

In [ ]:
#correlations with agg__Activity_08_AWB45_025
dt_train_named6 = dt_train_named6.drop(columns=['agg__Activity_08_AWB45_045'])
#correlation with agg__question_Uitgebreid 
dt_train_named6 = dt_train_named6.drop(columns=['agg__Activity_04_BPT_010','agg__Activity_04_BPT_020'])

In [ ]:
#The activity 330 stays, the 2 others need to be dropped
correlations4 = correlations4.drop(columns=['agg__Activity_01_HOOFD_370','agg__Activity_01_HOOFD_195','agg__Activity_01_HOOFD_375','agg__Activity_01_HOOFD_380','agg__Activity_01_HOOFD_430','agg__Activity_09_AH_I_010'])
#correlations with agg__Activity_08_AWB45_045
correlations4 = correlations4.drop(columns=['agg__Activity_08_AWB45_045'])
#correlation with agg__question_Uitgebreid 
correlations4 = correlations4.drop(columns=['agg__Activity_04_BPT_010','agg__Activity_04_BPT_020'])

In [ ]:
columns = list(correlations4.columns)

In [ ]:
correlations4_1 = dt_train_named6[columns]

cor = correlations4_1.corr()

cor[cor>0.6]

In [ ]:
#print the remaining columns

In [ ]:
columns

In [ ]:
len(columns)

In [ ]:
columns = ['agg__Activity_08_AWB45_020_2',
 'static__Milieu (vergunning)',
 'agg__question_28',
 'agg__monitoringResource_4634935',
 'agg__Activity_08_AWB45_025',
 'agg__question_Uitgebreid',
 'agg__Activity_08_AWB45_170',
 'agg__Activity_01_HOOFD_330',
 'agg__org:resource_560530',
 'agg__org:resource_4634935',
 'agg__Activity_01_HOOFD_193',
 'agg__question_other']

In [ ]:
for i in columns:
    print(i)

In [ ]:
dt_train_named[columns].nunique()

In [ ]:
for i in columns:
    print(i)

##### correlations (>0.15 with label)

In [ ]:
cor = dt_train_named6.corr()

cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.15].sort_values(ascending=False)
relevant_features.sort_values(ascending=False)

In [ ]:
#A lot of relevant features, so you better put these in a list
columns = list(relevant_features.index)
correlations5 = dt_train_named6[columns]

cor = correlations5.corr()

cor[cor>0.6]

In [ ]:
#correlations with question uitgebreid
dt_train_named7 = dt_train_named6.drop(columns=['agg__Activity_04_BPT_030'])

In [ ]:
#correlations with agg__Activity_01_HOOFD_330
dt_train_named7 = dt_train_named7.drop(columns=['agg__Activity_01_HOOFD_480'])
dt_train_named7 = dt_train_named7.drop(columns=['agg__Activity_01_HOOFD_200'])

In [ ]:
#correlates with 060
dt_train_named7 = dt_train_named7.drop(columns=['agg__Activity_08_AWB45_070_1'])
dt_train_named7 = dt_train_named7.drop(columns=['agg__Activity_08_AWB45_070_2'])

In [ ]:
#490_1
dt_train_named7 = dt_train_named7.drop(columns=['agg__Activity_01_HOOFD_490_2'])

In [ ]:
#correlates with 191
dt_train_named7 = dt_train_named7.drop(columns=['agg__Activity_01_HOOFD_192'])

In [ ]:
#250
dt_train_named7 = dt_train_named7.drop(columns=['agg__Activity_01_HOOFD_260'])

In [ ]:
#timesincecasestartsum
dt_train_named7 = dt_train_named7.drop(columns=['agg__timesincecasestart_max'])

##### correlations (>0.10 with label)

In [ ]:
cor = dt_train_named7.corr()

In [ ]:
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.10].sort_values(ascending=False)
relevant_features = relevant_features.sort_values(ascending=False)
relevant_features

In [ ]:
#A lot of relevant features, so you better put these in a list
columns = list(relevant_features.index)
correlations5 = dt_train_named7[columns]

In [ ]:
cor = correlations5.corr()
cor[cor>0.6]

In [ ]:
for i in range(0,len(cor)):
    for j in range(i,len(cor)):
        if i != j:
            if cor.iloc[i,j]>0.6:
                print(correlations5.dtypes.index[i],correlations5.columns[j], cor.iloc[i,j])

In [ ]:
relevant_features

In [ ]:
dt_train_named8 = dt_train_named7.drop(columns=['agg__timesincecasestart_sum'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__timesincecasestart_mean'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__timesincecasestart_std'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__Activity_08_AWB45_070_3'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__Activity_08_AWB45_090_1'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__timesincelastevent_mean'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__timesincelastevent_std'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__timesincelastevent_max'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__Activity_01_HOOFD_250_2'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__Activity_01_HOOFD_190_2'])
dt_train_named8 = dt_train_named8.drop(columns=['agg__weekday_max'])

In [ ]:
correlations5 = correlations5.drop(columns=['agg__timesincecasestart_sum'])
correlations5 = correlations5.drop(columns=['agg__timesincecasestart_mean'])
correlations5 = correlations5.drop(columns=['agg__timesincecasestart_std'])
correlations5 = correlations5.drop(columns=['agg__timesincelastevent_mean'])
correlations5 = correlations5.drop(columns=['agg__timesincelastevent_std'])
correlations5 = correlations5.drop(columns=['agg__timesincelastevent_max']) 
correlations5 = correlations5.drop(columns=['agg__Activity_08_AWB45_070_3']) 
correlations5 = correlations5.drop(columns=['agg__Activity_08_AWB45_090_1']) 
correlations5 = correlations5.drop(columns=['agg__Activity_01_HOOFD_250_2']) 
correlations5 = correlations5.drop(columns=['agg__Activity_01_HOOFD_190_2']) 
correlations5 = correlations5.drop(columns=['agg__weekday_max']) 

In [ ]:
 relevant_features.index

In [ ]:
columns = ['agg__Activity_08_AWB45_020_2', 'static__Milieu (vergunning)',
       'agg__question_28', 'agg__monitoringResource_4634935',
       'agg__Activity_08_AWB45_025', 'agg__question_Uitgebreid',
       'agg__Activity_08_AWB45_170', 'agg__Activity_01_HOOFD_330',
       'agg__org:resource_560530', 'agg__org:resource_4634935',
       'agg__Activity_01_HOOFD_193', 'agg__question_other',
       'agg__Activity_08_AWB45_060', 'agg__Activity_01_HOOFD_191',
       'agg__Activity_08_AWB45_051_0', 'agg__timesincelastevent_sum',
       'agg__Activity_01_HOOFD_490_1', 'agg__Activity_01_HOOFD_250',
       'agg__Activity_08_AWB45_090_2', 'static__Kap',
       'static__Milieu (neutraal wijziging)', 'agg__Activity_08_AWB45_070_3',
       'static__Gebiedsbescherming', 'agg__month_std',
       'agg__timesincecasestart_std', 'agg__Activity_01_HOOFD_250_1',
       'agg__question_42', 'agg__Activity_01_HOOFD_130',
       'agg__Activity_01_HOOFD_250_2', 'agg__weekday_std',
       'agg__Activity_08_OLO_100', 'agg__Activity_01_HOOFD_190_1',
       'static__Responsible_actor_560530', 'agg__Activity_11_AH_II_010',
       'agg__Activity_08_AWB45_050', 'agg__timesincecasestart_mean',
       'agg__weekday_max', 'agg__timesincelastevent_max',
       'agg__Activity_08_AWB45_090_1', 'agg__timesincelastevent_std',
       'agg__Activity_08_AWB45_020_0', 'agg__timesincecasestart_sum',
       'agg__open_cases_std', 'agg__timesincelastevent_mean',
       'agg__Activity_01_HOOFD_190_2', 'static__Inrit/Uitweg',
       'agg__Activity_01_HOOFD_470', 'agg__Activity_01_HOOFD_510_2',
       'agg__month_min', 'agg__Activity_01_HOOFD_495']

## Dataset traffic_fines_1

In [ ]:
dataset_ref = 'traffic_fines_1'
params_dir = './params_dir'
results_dir = './results' 
cls_encoding = 'agg'

gap = 1
n_iter = 1

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]

In [ ]:
# print dataset name
for dataset_name in datasets:
    print('Dataset:', dataset_name)

In [ ]:
# read the data
dataset_manager = DatasetManager(dataset_name)
data = dataset_manager.read_dataset()
cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

In [ ]:
#dimensions of the data
print('#columns:',len(data.columns))
print('#rows:',data.shape[0])

In [ ]:
#data types
data.dtypes

In [ ]:
#data description

This event log originates from an Italian local police force, containing events about notes sent about a fine.

### correlations

#### original dataset

##### load xes file

In [ ]:
#original dataset
#this is useful for the original activity names

import pm4py
log = pm4py.read_xes('Road_Traffic_Fine_Management_Process.xes.gz')

import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
dataframe = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)

pd.set_option("display.max_columns", dataframe.columns.shape[0])
pd.set_option("display.max_rows", dataframe.index.shape[0])



In [ ]:
dataframe.dtypes

In [ ]:
dataframe.head()

In [ ]:
#show the original activities names next to the new activity names

dataframe.groupby(['concept:name']).size()

##### correlations

In [ ]:
#check the correlation with Pearson's correlation for the dataset
data2 = data.copy()
data2.head()

In [ ]:
data2.label.head()

In [ ]:
data2.label.unique()
data2["label"].replace({"regular": 0, "deviant": 1}, inplace=True)
data2.label.unique()

In [ ]:
data2.dtypes

#### Transformed dataset

In [ ]:
# determine min and max (truncated) prefix lengths

min_prefix_length = 1
if "traffic_fines" in dataset_name:
    max_prefix_length = 10
elif "bpic2017" in dataset_name:
    max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
else:
    max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

# split into training and test
train, test = dataset_manager.split_data_strict(data, train_ratio, split="temporal")

In [ ]:
#prefix generation of training data
dt_train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length) 
#get the label of the train set
train_y = dataset_manager.get_label_numeric(dt_train_prefixes)   

In [ ]:
#transform train dataset and add the column names back to the dataframe
feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])
feature_combiner.fit(dt_train_prefixes, train_y)
dt_train_prefixes = feature_combiner.transform(dt_train_prefixes)
dt_train_prefixes = pd.DataFrame(dt_train_prefixes)
dt_train_named = dt_train_prefixes.copy()
names= feature_combiner.get_feature_names()
dt_train_named.columns = names

#concat them back to correlate correlations

train_y = pd.DataFrame(train_y)
train_y  = train_y.rename(columns={train_y.columns[0]:'label'})
dt_train_named2 = pd.concat([dt_train_named,pd.DataFrame(train_y)], axis=1)

##### correlations (>0.15 with label)

In [ ]:
#check the correlations

#Using Pearson Correlation
cor = dt_train_named2.corr()

In [ ]:
#correlation with label
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>.15]
relevant_features = relevant_features.sort_values(ascending=False)
relevant_features

In [ ]:
columns = list(relevant_features.index)

In [ ]:
columns

In [ ]:
#this is the dataframe with the features you want to check if they are correlated with eachother
correlations = dt_train_named2[columns]

In [ ]:
corr = correlations.corr()

corr[corr>0.6]

In [ ]:
#correlates with agg_Activity_send_fine

In [ ]:
correlations = correlations.drop(columns=['agg__expense_max','agg__expense_std','agg__expense_mean','agg__Activity_Insert Fine Notification','agg__timesincelastevent_mean'])

In [ ]:
#correlates with agg_Activity_send_fine

In [ ]:
correlations = correlations.drop(columns=['agg__month_std'])

In [ ]:
dt_train_named3 = dt_train_named2.drop(columns=['agg__expense_max','agg__expense_std','agg__expense_mean','agg__Activity_Insert Fine Notification','agg__timesincelastevent_mean','agg__month_std'])

In [ ]:
list(correlations.columns)

##### correlations (>0.10 with label)

In [ ]:
#check the correlations

#Using Pearson Correlation
cor = dt_train_named3.corr()

In [ ]:
#correlation with label
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>.10]
relevant_features = relevant_features.sort_values(ascending=False)
relevant_features

In [ ]:
columns = list(relevant_features.index)

In [ ]:
#this is the dataframe with the features you want to check if they are correlated with eachother
correlations2 = dt_train_named3[columns]

In [ ]:
corr = correlations2.corr()

corr[corr>0.6]

In [ ]:
relevant_features

In [ ]:
correlations2 = correlations2.drop(columns=['agg__open_cases_std','agg__timesincelastevent_std','agg__timesincecasestart_std'])

In [ ]:
list(correlations2.columns)

## Dataset BPIC17_0_Accepted

In [ ]:
dataset_ref = 'bpic2017_accepted'
params_dir = './params_dir'
results_dir = './results' 
cls_encoding = 'agg'

gap = 1
n_iter = 1

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]

In [ ]:
# print dataset name

for dataset_name in datasets:
    print('Dataset:', dataset_name)

# read the data

dataset_manager = DatasetManager(dataset_name)
data = dataset_manager.read_dataset()
cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

In [ ]:
#dimensions of the data

print('#columns:',len(data.columns))
print('#rows:',data.shape[0])

In [ ]:
#data types
data.dtypes

In [ ]:
#data description

This event log originates from the same financial institution as the BPIC2012 one.
However, the data collection has been improved, resulting in a richer and cleaner dataset. As in the previous case, the event log records execution traces of a loan application process. Similarly to BPIC2012, we define three separate labelings based on the outcome of the application, referred to

### correlations

#### original dataset

In [ ]:
#check the correlation with Pearson's correlation for the dataset
data2 = data.copy()
data2.head()

data2.label.head()

data2.label.unique()
data2["label"].replace({"regular": 0, "deviant": 1}, inplace=True)
data2.label.unique()

#### transformed dataset

In [ ]:
cls_method ='llm'

In [ ]:
# determine min and max (truncated) prefix lengths

min_prefix_length = 1
if "traffic_fines" in dataset_name:
    max_prefix_length = 10
elif "bpic2017" in dataset_name:
    max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
else:
    max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

# split into training and test
train, test = dataset_manager.split_data_strict(data, train_ratio, split="temporal")

#prefix generation of training data

dt_train_prefixes = dataset_manager.generate_prefix_data(train, min_prefix_length, max_prefix_length) 
#get the label of the train set
train_y = dataset_manager.get_label_numeric(dt_train_prefixes)   

In [ ]:
#transform train dataset and add the column names back to the dataframe
feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])
feature_combiner.fit(dt_train_prefixes, train_y)
dt_train_named = feature_combiner.transform(dt_train_named)
dt_train_named = pd.DataFrame(dt_train_named)
dt_train_named = dt_train_prefixes.copy()
names= feature_combiner.get_feature_names()
dt_train_named.columns = names

#concat them back to correlate correlations

train_y = pd.DataFrame(train_y)
train_y  = train_y.rename(columns={train_y.columns[0]:'label'})
dt_train_named2 = pd.concat([dt_train_named,pd.DataFrame(train_y)], axis=1)

##### correlations (>0.2 with label)

In [ ]:
#Using Pearson Correlation
cor = dt_train_named2.corr()

In [ ]:
#correlation with label

cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.2]
relevant_features = relevant_features.sort_values(ascending=False)
relevant_features

In [ ]:
columns = list(relevant_features.index)
#this is the dataframe with the features you want to check if they are correlated with eachother
correlations = dt_train_named2[columns]

In [ ]:
corr = correlations.corr()
corr[corr>0.6]

In [ ]:
#correlated with agg__CreditScore_std

In [ ]:
correlations2 = correlations.drop(columns=['agg__Selected_True','agg__CreditScore_max','agg__CreditScore_mean','agg__CreditScore_sum'])

In [ ]:
dt_train_named3 = dt_train_named2.drop(columns=['agg__Selected_True','agg__CreditScore_max','agg__CreditScore_mean','agg__CreditScore_sum'])

In [ ]:
list(correlations2.columns)

##### correlations (0.15> with label)

In [ ]:
#Using Pearson Correlation
cor = dt_train_named3.corr()

In [ ]:
#correlation with label
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>.15]
relevant_features = relevant_features.sort_values(ascending=False)
relevant_features

In [ ]:
columns = list(relevant_features.index)

In [ ]:
#this is the dataframe with the features you want to check if they are correlated with eachother
correlations = dt_train_named3[columns]

In [ ]:
corr = correlations.corr()
corr[corr>0.6]

In [ ]:
list(correlations.columns)

##### correlations (>0.10 with label)

In [ ]:
#correlation with label
cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>.10]
relevant_features = relevant_features.sort_values(ascending=False)
relevant_features

In [ ]:
columns = list(relevant_features.index)

In [ ]:
correlations2 = dt_train_named3[columns]

In [ ]:
cor = correlations2.corr()

In [ ]:
cor[cor > 0.6]

In [ ]:
#correlates with activity submitted

In [ ]:
correlations2 = correlations2.drop(columns=['agg__Activity_W_Handle leads','agg__org:resource_User_1'])

In [ ]:
dt_train_named4 = dt_train_named3.drop(columns=['agg__Activity_W_Handle leads','agg__org:resource_User_1'])

In [ ]:
list(correlations2.columns)

##### even more

In [ ]:
#correlation with label

cor_target = abs(cor["label"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>.08]
relevant_features.sort_values(ascending=False)

In [ ]:
columns = list(relevant_features.index)

In [ ]:
correlations5 = dt_train_named5[columns]

In [ ]:
cor = correlations5.corr()

In [ ]:
cor[cor>.6]

In [ ]:
dt_train_named6 = dt_train_named5.drop(columns=['agg__Activity_A_Validating','agg__NumberOfTerms_mean','agg__Activity_W_Call after offers'])

In [ ]:
correlations5 = correlations5.drop(columns=['agg__Activity_W_Call after offers'])

In [ ]:
columns = list(correlations5.columns)

In [ ]:
columns

In [ ]:
columns = ['static__ApplicationType_Limit raise',
 'static__ApplicationType_New credit',
 'agg__Activity_A_Cancelled',
 'agg__Activity_A_Submitted',
 'agg__Activity_W_Validate application',
 'agg__lifecycle:transition_ate_abort',
 'agg__Accepted_False',
 'agg__Accepted_True',
 'agg__Selected_False',
 'agg__CreditScore_std']